### Установка

In [2]:
!pip install setfit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB

In [3]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Импорты

In [4]:
from utils import split_df_texts_by_sentences, choose_max_rate_sentence, text_accuracy_score
from sentence_transformers.losses import CosineSimilarityLoss
from sklearn.model_selection import train_test_split
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset
import pandas as pd
import numpy as np
import joblib
import json
import pickle
import gc

### Агрегация данных

In [ ]:
with open('../data/pickle_objects/contract_enforcement.pkl', 'rb') as fp:
  contract_enforcement = pickle.load(fp)
print(len(contract_enforcement))
contract_enforcement = pd.Series(contract_enforcement)

contract_enforcement = contract_enforcement[~contract_enforcement.isna()]
contract_enforcement = contract_enforcement.values
len(contract_enforcement)

988


984

In [ ]:
with open('../data/pickle_objects/garantee_enforcement.pkl', 'rb') as fp:
  garantee_enforcement = pickle.load(fp)
print(len(garantee_enforcement))
garantee_enforcement = pd.Series(garantee_enforcement)

garantee_enforcement = garantee_enforcement[~garantee_enforcement.isna()]
garantee_enforcement = garantee_enforcement.values
len(garantee_enforcement)

811


508

In [ ]:
with open('../data/pickle_objects/nonetype.pkl', 'rb') as fp:
  nonetype_sentences = pickle.load(fp)

In [ ]:
nonetype_sentences[:5]

array(['Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806',
       'Общая информация', 'Номер извещения 0328300032822000806',
       'Наименование объекта закупки', 'Поставка продуктов питания'],
      dtype='<U1071')

In [ ]:
len(nonetype_sentences)

46897

In [ ]:
nonetype_1500 = np.random.choice(nonetype_sentences, 1500)
df_for_contract_enforcement = pd.DataFrame(list(set(contract_enforcement)), columns=['sentence'])
df_for_nonetype = pd.DataFrame(nonetype_1500, columns=['sentence'])

In [ ]:
df_for_contract_enforcement['label'] = 1
df_for_nonetype['label'] = 0

In [ ]:
df_for_contract_enforcement.head()

,sentence,label
0,"В случае если Поставщиком, при участии в закуп...",1
1,В размере 5% от начальной максимальной цены До...,1
2,Поставщик внес обеспечение исполнения Контракт...,1
3,Размер обеспечения исполнения договора составл...,1
4,Размер обеспечения исполнения договора 5 % от ...,1


In [ ]:
df_for_nonetype.head()

,sentence,label
0,Восход. ие об одностороннем отказе от исполнен...,0
1,Оплата Стороной неустойки (штрафа,0
2,1. ОБЕСПЕЧЕНИЕ ИСПОЛНЕНИЯ КОНТРАКТА,0
3,Кущевская «__»__________2022 г.,0
4,Требования к гарантии качества товара,0


In [ ]:
df_contract_enforcement_model = pd.concat([df_for_contract_enforcement, df_for_nonetype], axis=0)

In [ ]:
df_contract_enforcement_model = df_contract_enforcement_model.sample(frac=1).reset_index(drop=True)

In [ ]:
nonetype_1500 = np.random.choice(nonetype_sentences, 1500)
df_for_garantee_enforcement = pd.DataFrame(list(set(garantee_enforcement)), columns=['sentence'])
df_for_nonetype = pd.DataFrame(nonetype_1500, columns=['sentence'])

df_for_garantee_enforcement['label'] = 1
df_for_nonetype['label'] = 0

df_for_garantee_enforcement_model = pd.concat([df_for_garantee_enforcement, df_for_nonetype], axis=0)



In [ ]:
df_for_garantee_enforcement_model = df_for_garantee_enforcement_model.sample(frac=1).reset_index(drop=True)


In [ ]:
df_for_garantee_enforcement_model.head()

,sentence,label
0,Гарантии качества распространяются на все конс...,0
1,Коэффициент снижения (К) рассчитывается по фор...,0
2,Размер обеспечения гарантийных обязательств со...,1
3,"Порядок обеспечения исполнения контракта, треб...",0
4,Гарантийный срок на технологическое и инженерн...,0


### SetFit model

In [ ]:
train, test = train_test_split(df_contract_enforcement_model, test_size=0.98)

In [ ]:
len(train), len(test)

(31, 1567)

In [ ]:
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    column_mapping={"sentence": "text", "label": "label"},
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
trainer.train()
metrics = trainer.evaluate()

Applying column mapping to training dataset
***** Running training *****
  Num examples = 1240
  Num epochs = 1
  Total optimization steps = 78
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78 [00:00<?, ?it/s]

Applying column mapping to evaluation dataset
***** Running evaluation *****


Так как модель SetFit отлично работает на малом количестве данных, можем оставить основную часть датасета в валидационной выборке

In [ ]:
metrics

{'accuracy': 0.982769623484365}

In [ ]:
del trainer

In [ ]:
gc.collect()


217

модель показывает accuracy 0.98 на наборе из 31 строки данных. 


### Batch size = 3
попробуем уменьшить размер батча, чтобы не перегружать ОЗУ

In [ ]:
train, test = train_test_split(df_contract_enforcement_model, test_size=0.94)

In [ ]:
len(train), len(test)

(125, 1973)

In [ ]:
del df_for_garantee_enforcement_model, df_for_nonetype, nonetype_1500, df_contract_enforcement_model, nonetype_sentences, garantee_enforcement, contract_enforcement
gc.collect()

15

In [ ]:
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    batch_size=3,
    column_mapping={"sentence": "text", "label": "label"},
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
trainer.train()
metrics = trainer.evaluate()

Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 5000
  Num epochs = 1
  Total optimization steps = 1667
  Total train batch size = 3


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1667 [00:00<?, ?it/s]

Applying column mapping to evaluation dataset
***** Running evaluation *****


In [ ]:
metrics

{'accuracy': 0.9655347187024835}

In [ ]:
model.predict_proba(['Размер обеспечения 1000 рублей'])

tensor([[0.0166, 0.9834]], dtype=torch.float64)

In [ ]:
joblib.dump(model, '../data/pickle_objects/contract_enforcement_setfit_model.joblib')

['contract_enforcement_setfit_model.joblib']

In [ ]:
# joblib.dump(model, '/content/drive/MyDrive/забег/contract_enforcement_setfit_model.joblib')


['/content/drive/MyDrive/забег/contract_enforcement_setfit_model.joblib']

['/content/drive/MyDrive/забег/contract_enforcement_setfit_model.joblib']

In [ ]:
# from google.colab import files
# files.download('contract_enforcement_setfit_model.joblib')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Применение моделей на неразмеченных данных

так как модель получает на вход целые предложения, без особой предобработки, алгоритм выглядит так:


1.   Делим текст на предложения
2.   В зависимости от label отправляем в цикле предложения текста в соответствующуюю модель.
3.   Выбираем предложение с наибольшим скором



In [5]:
contract_enforcement_model = joblib.load('../data/models/contract_enforcement_setfit_model.joblib')
garantee_enforcement_model = joblib.load('../data/models/garantee_enforcement_setfit_model.joblib')

In [25]:
# contract_enforcement_model = joblib.load('/content/drive/MyDrive/забег/contract_enforcement_setfit_model.joblib')
# garantee_enforcement_model = joblib.load('/content/drive/MyDrive/забег/garantee_enforcement_setfit_model.joblib')

In [36]:
def get_contract_enforcement_best_sentence(text:list, max_val=0.7):
  result = {}
  for sentence in text:
    if sentence:
      res = contract_enforcement_model.predict_proba([sentence])[0][1].item()
      result[sentence] = res

  sentence, value = choose_max_rate_sentence(result)

  if value > max_val:
    return sentence, value
  return None, None

In [37]:
def get_garantee_enforcement_best_sentence(text:list, max_val):
  result = {}
  for sentence in text:
    if sentence:
      res = garantee_enforcement_model.predict_proba([sentence])[0][1].item()
      result[sentence] = res

  sentence, value = choose_max_rate_sentence(result)

  if value > max_val:
    return sentence, value
  return None, None

In [38]:
def predict(dataframe:pd.DataFrame, max_val):
  predictions = []
  # делим датафрейм на предложения 
  df_splitted_by_sentences = split_df_texts_by_sentences(dataframe.copy())
  lenght = len(df_splitted_by_sentences)

  # проходим в циклe, используем соответствующую модель
  for i in range(lenght):
    if df_splitted_by_sentences.loc[i, 'label'] == 'обеспечение исполнения контракта':
      text = df_splitted_by_sentences.loc[i, 'text']
      best_sentence, val = get_contract_enforcement_best_sentence(text)

    elif df_splitted_by_sentences.loc[i, 'label'] == 'обеспечение гарантийных обязательств':
      text = df_splitted_by_sentences.loc[i, 'text']
      best_sentence, val = get_garantee_enforcement_best_sentence(text, max_val)

    else: 
      print('Неверный формат label')
    print(f'{i} из {lenght} предсказано\n {best_sentence} score: {val}\n{best_sentence==df_splitted_by_sentences.loc[i, "target"]}\n {df_splitted_by_sentences.loc[i, "target"]}')
    predictions.append(best_sentence)

  return predictions

In [29]:
df = pd.read_json('../data/train_unpacked.json', encoding='utf-8')
df.sample(3)

,id,text,label,target
817,561226408,3 УТВЕРЖДЕНО приказом филиала АО «Мособлгаз» «...,обеспечение исполнения контракта,Сумма обеспечения исполнения договора предусмо...
1648,586142786,Государственное автономное учреждение Республи...,обеспечение гарантийных обязательств,Обеспечение гарантийных обязательств установле...
1641,30093679,ДОГОВОР № _________ на выполнение комплекса ра...,обеспечение гарантийных обязательств,None


Для замера accuracy для уменьшения времени работы (датасет из 1799 объектов пресказывается 12 часов на колабе) возьмём 500 объектов. 

In [30]:
df_500 = df.sample(500)
df_500.reset_index(drop=True, inplace=True)

In [31]:
df_500.head()

,id,text,label,target
0,214648417,УТВЕРЖДЕНА приказом Федеральной службы по экол...,обеспечение гарантийных обязательств,None
1,445125138,Федеральное государственное бюджетное образова...,обеспечение гарантийных обязательств,None
2,504104789,Федеральное государственное унитарное предприя...,обеспечение гарантийных обязательств,None
3,110916274,«УТВЕРЖДАЮ» Заместитель генерального директора...,обеспечение исполнения контракта,Размер обеспечения исполнения договора: 5 % от...
4,39069440,«Проект договора» Договор №_____ на поставку а...,обеспечение гарантийных обязательств,Обеспечение гарантийных обязательств устанавли...


In [34]:
pred = predict(df_500, 0.93)

0 из 500 предсказано
 УТВЕРЖДЕНА приказом Федеральной службы по экологическому, технологическому и атомному надзору от 4 марта 2019 г. score: 0.9928834958858669
False
 None
1 из 500 предсказано
 None score: None
True
 None
2 из 500 предсказано
 Руководитель ГЦЗ отд. 055 ________________ С.Г. score: 0.9924874849784977
False
 None
3 из 500 предсказано
 Размер обеспечения заявок на участие в электронном аукционе: 1% от начальной (максимальной) цены договора score: 0.9870363977528901
False
 Размер обеспечения исполнения договора: 5 % от начальной (максимальной) цены договора
4 из 500 предсказано
 Обеспечение гарантийных обязательств устанавливается в размере 1 % начальной (максимальной) цены контракта. score: 0.9935411324535374
True
 Обеспечение гарантийных обязательств устанавливается в размере 1 % начальной (максимальной) цены контракта.
5 из 500 предсказано
 Размер обеспечения исполнения договора составляет 5 % от начальной (максимальной) цены договора – 4 450 (четыре тысячи четыреста п

In [35]:
text_accuracy_score(pred, df_500.loc[:, 'target'].values)


0.55

accurcy 0.55 для сравнения предскажем тоже самое с помощью xgboost. на том же самом датасете xgboost показывает accuracy 0.62